In [1]:
# Pacotes dash, pandas, plotly e numpy

import dash
from dash import dcc
from dash import html
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import numpy as np

In [2]:
df = pd.read_csv('gapminder_alcohol.csv', encoding='latin-1')
df_country = pd.read_csv('countryContinent.csv', encoding='latin-1')

In [3]:
df_country.drop(['code_2', 'country_code', 'iso_3166_2', 'sub_region', 'region_code', 'sub_region_code'], axis='columns', inplace=True)
df_country = df_country.rename(columns={'country': 'País', 'continent': 'Continente'})
df_country.replace('Europe', 'Europa', inplace=True)

In [4]:
df = df.rename(columns={'country': 'País', 'alcconsumption': 'Consumo alcoolico', 'incomeperperson': 'Renda', 'suicideper100th': 'Taxa de suicidio', 'employrate': 'Taxa de emprego', 'urbanrate': 'População urbana' })

In [5]:
df = df.dropna(subset=["Consumo alcoolico"])


In [6]:
df = pd.merge(df, df_country, how = 'left', on='País')

In [7]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWKwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

In [8]:
! ngrok authtoken 24ZkCk77qCZZtOv5seygZ8VTdqT_6hKAJbCuRNx3wCLiZ9tTu

Authtoken saved to configuration file: C:\Users\lucas/.ngrok2/ngrok.yml


In [9]:
from pyngrok import ngrok

public_url = ngrok.connect('8050')
public_url

<NgrokTunnel: "http://624d-177-37-203-86.ngrok.io" -> "http://localhost:8050">

In [ ]:
indicadores_lista = ['Consumo alcoolico', 'Renda', 'Taxa de suicidio', 'Taxa de emprego', 'População urbana']
continentes_lista = df['Continente'].unique()
mapa = px.choropleth(df, locations="code_3",
                    color='Consumo alcoolico', # lifeExp is a column of gapminder
                    hover_name="País", # column to add to hover information
                    color_continuous_scale='reds')

app.layout = html.Div([
    
    html.H1(
        children='Dashboard - Indicadores sociais e consumo de bebidas alcoolicas',
        style={
            'textAlign': 'center'
        }
    ),
    
    html.H4(
        children='''
            Renda: Produto Interno Bruto per capita em US$ 2.000 constantes.
            *OBS: As diferenças no custo de vida entre os países,
            foram levadas em consideração
        '''),
    
    html.H4(
        children='''
            Consumo alcoolico: Consumo per capita em litros de álcool puro (idade: 15+)
        '''),
    
    html.H4(
        children='''
            Taxa de suicidio: Suicídio, ajustado por idade, por 100.000 habitantes
            padrão devido a mortalidade por lesão autoinfligida.
        '''),
    
    html.H4(
        children='''
            Taxa de emprego: Percentagem da população total, com idade superior a 15 anos,
            que esteve empregada durante o ano determinado.
        '''),
    
    html.H4(
        children='''
            População urbana: População que vive em áreas urbanas.
        '''),
    
    html.H3(children='Consumo de bebidas alcoolicas no mundo:', style={
        'textAlign': 'center'
    }),
    
     dcc.Graph(
        id='graph',
        figure=mapa
    ),
    
    html.H3(children='Indicadores:', style={
        'textAlign': 'center'
    }),
    
    html.Div([
        dcc.Dropdown(
            id='indicador_filtro',
            options=[{'label':i, 'value':i} for i in indicadores_lista],
            value='Renda')
    ],style = {'width': '100%', 'align-items': 'center', 'justify-content': 'center'}),    
    
    html.Div([
        dcc.Graph(id='graph1')
    ], style={'width': '49%', 'float': 'left', 'display': 'inline-block'}),
    
    html.Div([
        dcc.Graph(id='graph2')
    ], style={'width': '49%', 'float': 'right','display': 'inline-block'}),
    
    html.H3(children='Visão detalhada do continente:', style={
        'textAlign': 'center'
    }),
    
    html.Div([
        dcc.Dropdown(
            id='continente_filtro',
            options=[{'label':i, 'value':i} for i in continentes_lista],
            value='Asia')
    ],style = {'width': '100%', 'align-items': 'center', 'justify-content': 'center'}),   
    
    html.Div([
        dcc.Graph(id='graph3')
    ], style={'width': '100%', 'display': 'inline-block'})           
    
])

@app.callback(
    dash.dependencies.Output('graph1', 'figure'),
    dash.dependencies.Output('graph2', 'figure'),    
    dash.dependencies.Output('graph3', 'figure'),
    dash.dependencies.Input('indicador_filtro', 'value'),
    dash.dependencies.Input('continente_filtro', 'value'))

def update_graph(indicador_filtro, continente_filtro):
    fig = px.scatter(df, x='Consumo alcoolico', y=indicador_filtro, color='Continente')
    
    fig2 = px.pie(df, values=indicador_filtro, names='Continente', title=indicador_filtro)

    fig3 = px.sunburst(df[df.Continente == continente_filtro], path=['País'], values='Consumo alcoolico',
                      color='Consumo alcoolico', hover_data=['Taxa de suicidio'])
    
    return fig, fig2, fig3

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [ ]:
ngrok.kill()

In [ ]:
fig2 = go.Figure(data = go.Choropleth(
    locations = df['code_3'],
    z = df['Consumo alcoolico'],
        text = df['País'],
        colorscale = 'reds',
        autocolorscale = False,
        reversescale = False,
        marker_line_color = 'white',
        marker_line_width = .5,
        colorbar_title = 'Consumo alcoolico'))

fig2.show()

In [ ]:
fig = px.sunburst(df[df.Continente == 'Asia'], path=['País'], values='Consumo alcoolico',
                  color='Consumo alcoolico', hover_data=['Taxa de suicidio'])
fig.show()

In [ ]:
fig = px.scatter(df, y="Taxa de suicidio", x="Consumo alcoolico", color="Continente")
#fig.show()
app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),
    
    html.Div(children='''
        Dash: A web application framework for Python
    '''),
    
    dcc.Graph(
        id='graph1',
        figure=fig
    )
])

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)